In [26]:
import numpy as np
from scipy.optimize import minimize
import pandas as pd
import matplotlib.pyplot as plt
import os

In [27]:
def current(B, arrJ, arrC, y): # y is initial phase difference of the whole circuit, B is the magnetic field, arrJ is the location of junctions, arrC is critical current associated with each junction
    curr = 0 # summation of all currents in the entire junction
    limit = int(len(arrJ) / 2) # number of junctions in the SQUID
    numOfSegments = 0
    for n in range(limit):
        numOfSegments = 5 if (arrJ[2 * n + 1] - arrJ[2 * n]) < 0.05 else int(100 * (arrJ[2 * n + 1] - arrJ[2 * n]))
        sizeOfSegment = float((arrJ[2 * n + 1] - arrJ[2 * n]) / numOfSegments)
        for i in range(numOfSegments):
            curr += arrC[n] * np.sin(y + (2 * np.pi * B) * (arrJ[2 * n] + i * sizeOfSegment)) * (1/numOfSegments)

    # phase difference evolves according to 2 * pi * B

    # curr += (critical current element in array)(sin(y + (2 * pi * B) * length)
    return curr

def maxCurrent(B, arrayJ, arrayC): # Spits out the maximum current by varying the gauge invariant phase of the left end (free parameter) gamma
    Y=np.linspace(0, 2*np.pi, 150)
    dummyArray=[]
    for gamma in Y:
        dummyArray.append(current(B, arrayJ, arrayC, gamma))
    return max(dummyArray)

def criticalCurrent(density, arrJ):
    criticalCurrents = []
    junctionWidths = []
    for i in range(len(arrJ)//2):
        junctionWidths.append(arrJ[2*i+1] - arrJ[2*i])
    for i in range (len(junctionWidths)):
        criticalCurrents.append(junctionWidths[i] * density[i])
    return criticalCurrents, junctionWidths

In [28]:
def cost_function(arr):
    generated_data = []
    model_data_csv = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
    model_i_c = np.array(model_data_csv['I_c'])
    model_B = np.array(model_data_csv['B'])
    criticalCurrents, junctionWidths = criticalCurrent(density=np.ones(len(arr)//2), arrJ=arr)
    for b in model_B:
        generated_data.append(maxCurrent(B=b, arrayJ=arr, arrayC=criticalCurrents))
    diff = model_i_c - generated_data
    for i in range(len(diff)):
        diff[i] = diff[i]**2
    return np.sum(diff)

def optimize_array(n):
    initial_guess = np.linspace(0, 1, n)
    initial_guess[0] = 0
    initial_guess[-1] = 1
    bounds = [(0, 1)] * n
    result = minimize(cost_function, initial_guess, method='BFGS', bounds=bounds, tol=1e-2)
    return result.x, result.success

def array_to_python_syntax(arr):
    if not arr.any():
        return "[]"

    result = "["
    for item in arr:
        if isinstance(item, str):
            result += f"'{item}', "
        else:
            result += f"{item}, "
    
    # Remove the trailing comma and space
    result = result[:-2]
    result += "]"
    return result

def save_plot(x_values, y_values, directory, file_name):
    experimental_df = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
    experimental_x = np.array(experimental_df['B'])
    experimental_y = np.array(experimental_df['I_c'])

    # Plot the data
    plt.plot(x_values, y_values)
    plt.plot(experimental_x, experimental_y)
    plt.xlabel('B')
    plt.ylabel('I_c')

    # Ensure the directory exists
    os.makedirs(directory, exist_ok=True)

    # Save the plot
    plot_path = os.path.join(directory, file_name)
    plt.savefig(plot_path)

    # Close the plot to release resources
    plt.close()

def create_folder(repository_path, folder_name):
    folder_path = os.path.join(repository_path, folder_name)
    
    # Check if the folder already exists
    if not os.path.exists(folder_path):
        # Create the folder
        os.makedirs(folder_path)
        print(f"Folder '{folder_name}' created successfully.")
    else:
        print(f"Folder '{folder_name}' already exists.")
    return folder_path

def write_txt_file(directory, arrJ_gen, passed, file_name):
    converted_arrJ_gen = array_to_python_syntax(arrJ_gen)
    text_content = f"The generated Junction array is {converted_arrJ_gen}. \n \n Optimized?: {passed}"
    numOfJunctions = len(arrJ_gen)//2
    text_file_path = os.path.join(directory, file_name)
    with open(text_file_path, 'w') as file:
        file.write(text_content)

In [29]:
starting_junctions = 4
ending_junctions = 10
IMax = []
model_data_csv = pd.read_csv(r"experimental data\experimental csvs\5.0uV_normalized.csv")
model_i_c = np.array(model_data_csv['I_c'])
model_B = np.array(model_data_csv['B'])

for n in range(starting_junctions, ending_junctions):
    arrayOfJunctions, optimized = optimize_array(2 * n)
    criticalCurrents, junctionWidths = criticalCurrent(density=np.ones(len(arrayOfJunctions)//2), arrJ=arrayOfJunctions)
    sum = np.sum(criticalCurrents)
    for B in model_B:
        IMax.append(maxCurrent(B, arrayOfJunctions, criticalCurrents) / sum)
    directory = r"experimental data\candidate set-ups\BFGS optimized"
    general_name = f"{len(arrayOfJunctions)//2} Junctions"
    folder_path = create_folder(repository_path=directory, folder_name=general_name)
    write_txt_file(directory=folder_path, arrJ_gen=arrayOfJunctions, passed=optimized, file_name=general_name)
    save_plot(x_values=model_B, y_values=IMax, directory=folder_path, file_name=general_name)
    

C:\Users\nanowire\AppData\Local\Temp\ipykernel_18192\61492424.py:19: RuntimeWarning: Method BFGS cannot handle bounds.
  result = minimize(cost_function, initial_guess, method='BFGS', bounds=bounds, tol=1e-2)


KeyboardInterrupt: 

In [ ]:
for B in model_B:
        IMax.append(maxCurrent(B, arrayOfJunctions, criticalCurrents) / sum)
directory = r"experimental data\candidate set-ups\BFGS optimized"
general_name = f"{len(arrayOfJunctions)//2} Junctions"
folder_path = create_folder(repository_path=directory, folder_name=general_name)
write_txt_file(directory=folder_path, arrJ_gen=arrayOfJunctions, file_name=general_name)
save_plot(x_values=model_B, y_values=IMax, directory=folder_path, file_name=general_name)

Folder '3 Junctions' created successfully.
